# Target Detection for FMCW Radar

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc

## Radar parameters, constants, filenames, etc.

In [2]:
sample_rate = 10e6
buffer_size = 2**14
rx_lo_freq = 2e9
tx_lo_freq = 2e9
adc_size = 2**12

c = 3e8

nc_file = ''

## Noise calculation
### For coherent detectors, noise power can be estimated from the variance, $\sigma^2$,  (square of standard deviation)
$ \begin {align}
P_n &= \frac{\sigma^2}{2}
\end {align} $

## False alarm
### A false alarm occurs when noise jumps beyond the noise floor to a level similar to that of targets. There exists a voltage threshold, $V_T$, between a target and noise that is determined using SNR and determines the probability of false alarms, $P_{fa}$. 
$ \begin {align}
V_T &= \sqrt{-2\sigma^2 ln(P_{fa})} \\
P_d &= Q\left[\sqrt{2SNR}\sqrt{-2ln(P_{fa})}\right] \\
Q &- \text{Marcum's Q functions}
\end {align} $